In [ ]:
#default_exp labeling_icao

In [1]:
#export
from pathlib import Path
import numpy as np
from ipywidgets import widgets, Layout
from IPython.display import clear_output, display
import io
import json
from PIL import Image as PILImage

# Labeler

In [2]:
#export
LABELS_DIRPATH = Path('../data/custom_icao/labels/')
IMG_PATHS = [str(path) for path in Path('../data/face/unlabeled_cropped_images/').iterdir()]
IMG_SIZE = (100,100)
LABELS = [
    'MouthOpen',
    'Blink'
]

In [3]:
#export
def impath2bytes(path, newsize):
    img = PILImage.open(path, mode='r')
    img = img.resize(newsize)
    imgByteArr = io.BytesIO()
    img.save(imgByteArr, format='jpeg')
    return imgByteArr.getvalue()

In [4]:
#export
def make_button_widget(desc, handler=None, img_path=None, style=None, layout=Layout(width='auto'), color='blue'):
    if color == 'green': button_style = 'success'
    elif color == 'red': button_style = 'danger'
    else: button_style = 'info'
    if color == 'lightred': color = 'lightsalmon'
    btn = widgets.Button(description=desc, layout=layout, button_style=button_style)
    if color not in []: btn.style.button_color = color
    if handler is not None: btn.on_click(handler)
    if img_path is not None: btn.im_path = img_path
    if style is not None: btn.button_style = style
    return btn

def make_img_widget(impath, layout=Layout(height='250px', width='300px'), format='jpg'):
    return widgets.Image(value=impath2bytes(impath, IMG_SIZE), format=format, layout=layout)

def make_vertical_box(children, layout=Layout(width='auto', height='100px', overflow_x="hidden")):
    return widgets.VBox(children, layout=layout)

def make_horizontal_box(children, layout=Layout()):
    return widgets.HBox(children, layout=layout)

In [5]:
def make_img_widget(impath, layout=Layout(height='250px', width='300px'), format='jpg'):
    return widgets.Image(value=impath2bytes(impath, IMG_SIZE), format=format, layout=layout)

In [6]:
def make_vertical_box(children, layout=Layout(width='auto', height='100px', overflow_x="hidden")):
    return widgets.VBox(children, layout=layout)

In [7]:
def make_horizontal_box(children, layout=Layout()):
    return widgets.HBox(children, layout=layout)

In [8]:
#export
def change_btn_color(btn):
    value = 'green' if btn.style.button_color == 'red' else 'red'
    btn.style.button_color, btn.description = value, value

In [9]:
#export
def make_labeling_tab(desc, pred_score, true_score, pred_color, true_color):
    name_lbl = widgets.Label(desc, layout=Layout(left='0', height='25px'))
    pred_btn = make_button_widget(str(pred_score), layout=Layout(left='0', width='60px', height='25px'), color=pred_color)
    true_btn = make_button_widget(str(true_score), layout=Layout(left='0', height='25px'), color=true_color, handler=change_btn_color)
    return make_vertical_box([name_lbl, make_horizontal_box([pred_btn, true_btn])], layout=Layout(width='auto', height='60px', overflow_x="hidden"))

In [10]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) { return false; }

<IPython.core.display.Javascript object>

In [11]:
#export
def get_json_labels(path, labels=None):
    if not Path(path).exists(): return {lbl:None for lbl in labels}
    j = json.load(open(path))
    if labels is None: return j['labels']
    return {lbl:j['labels'].get(lbl) for lbl in labels}

In [12]:
#export
bool2clr = lambda boolean, mode='': mode+'green' if boolean else mode+'red'
round_nr = lambda f: int(f*100)

def save_json(dictionary, path):
    with open(path, 'w') as filepath:
        json.dump(dictionary, filepath)

class Labeler:
    def __init__(self, bin_names=LABELS, image_paths=IMG_PATHS, output_dir=LABELS_DIRPATH, bs=4, label_labeled=False):
        self.bs, self.bin_names, self.image_paths, self.output_dir, self.lbl_lbled = bs, bin_names, image_paths, output_dir, label_labeled
        
    def get_next_batch(self): # get images that are not already labeled (in labels dir)
        self.next_batch = []
        for path in map(Path, self.image_paths):
            if self.lbl_lbled or (path.name not in [x.name[:-5] for x in self.output_dir.iterdir()]): # [:-5] to remove '.json' end
                self.next_batch.append(path)
            self.image_paths = self.image_paths[1:]
            if len(self.next_batch) == self.bs: break
        return self.next_batch
        
    def show_next_batch(self):
        self.widget_list = []
        next_batch = self.get_next_batch()
        for path in next_batch:
            img_widget = make_img_widget(path, layout=Layout(height='200px', width='200px')) # 500 ?
            tabs = []
            labels = get_json_labels(self.output_dir/(path.name+'.json'), self.bin_names)
            for name, value in labels.items():
                tabs.append(make_labeling_tab(name, '', value, 'white', value))
            path_widget = widgets.Label(path.name, layout=Layout(left='0', height='25px'))
            self.widget_list.append( make_vertical_box([path_widget, img_widget] + tabs, layout=Layout(height='auto')) )
        display(make_horizontal_box(self.widget_list + [make_button_widget('next', layout=Layout(left='0'), handler=self.on_next_click)]))
        
    def on_next_click(self, btn):
        parsed_output = self.parse_widget_list()
        for path, labels in parsed_output:
            fn = Path(path).name + '.json'
            json = {'labels':labels, 'hash': hash(open(path))}
            save_json(json, self.output_dir/fn)
        clear_output()
        self.show_next_batch()
        
        
    def parse_widget_list(self): # returns list of (path, {'name':'red'/'green', 'name2':...}) tuple pairs (each elem is single image)
        out = []
        for widget, path in zip(self.widget_list, self.next_batch):
            filepath_widget, img_widget, *tabs = widget.children
            labels = {}
            for tab in tabs:
                name_widget, btns = tab.children
                name = str(name_widget.value)
                b1, b2 = btns.children
                label = b2.style.button_color
                labels[name] = label
            out.append((path,labels))
        return out

In [13]:
labeler = Labeler(LABELS, IMG_PATHS, LABELS_DIRPATH, bs=4, label_labeled=True)
labeler.show_next_batch()

In [105]:
bool2clr = lambda boolean, mode='': mode+'green' if boolean else mode+'red'
round_nr = lambda f: int(f*100)

In [111]:
# example:
json.loads(open(labeler.output_dir/'00077897.jpg.json').read())

{'labels': {'MouthOpen': 'green'}, 'hash': 8726196546821}